In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [13]:
DATA_PATH = os.path.join('dataset')
actions = np.array(['one', 'two','three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
sequence_length = 100
num_features = 126
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
sequences, labels = [], []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_path)).astype(int):
        frame_paths = [
            os.path.join(action_path, str(sequence), "{}.npy".format(frame_num))
            for frame_num in range(sequence_length)
        ]
        window = [np.load(frame_path) for frame_path in frame_paths]
        sequences.append(window)
        labels.append(label_map[action])

In [7]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [9]:
log_dir = os.path.join('VGG16_Logs')
callback = TensorBoard(log_dir=log_dir)
ACCURACY_THRESHOLD = 0.95

class MyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, monitor_metric='accuracy', threshold=0.95):
        super(MyCallback, self).__init__()
        self.monitor_metric = monitor_metric
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs={}): 
        current_metric_value = logs.get(self.monitor_metric)
        if current_metric_value is not None and current_metric_value > self.threshold:
            print(f"\nReached {self.threshold * 100:.2f}% {self.monitor_metric}, stopping training!")
            self.model.stop_training = True

tb_callback = TensorBoard(log_dir=log_dir)
my_callback = MyCallback(monitor_metric='accuracy', threshold=ACCURACY_THRESHOLD)

In [14]:
# Define the model
model = Sequential()

# Block 1
model.add(BatchNormalization())
model.add(Conv1D(64, 3, activation='relu', padding='same', input_shape=(sequence_length, num_features)))
model.add(MaxPooling1D(2, strides=2))

# Block 2
model.add(BatchNormalization())
model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Block 3
model.add(BatchNormalization())
model.add(Conv1D(256, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(256, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Block 4
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Block 5
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Flatten
model.add(Flatten())

# Fully Connected Layers
model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [17]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=8, callbacks=[tb_callback, my_callback, early_stopping])

Epoch 1/100
158/158 [==============================] - 14s 26ms/step - loss: 2.6373 - accuracy: 0.1754 - val_loss: 2.3057 - val_accuracy: 0.1833
Epoch 2/100
158/158 [==============================] - 3s 21ms/step - loss: 2.2755 - accuracy: 0.2921 - val_loss: 2.2725 - val_accuracy: 0.2759
Epoch 3/100
158/158 [==============================] - 3s 21ms/step - loss: 2.1758 - accuracy: 0.3556 - val_loss: 2.0576 - val_accuracy: 0.4019
Epoch 4/100
158/158 [==============================] - 3s 21ms/step - loss: 2.1748 - accuracy: 0.3643 - val_loss: 2.0784 - val_accuracy: 0.4111
Epoch 5/100
158/158 [==============================] - 3s 20ms/step - loss: 2.1519 - accuracy: 0.3675 - val_loss: 2.2018 - val_accuracy: 0.4685
Epoch 6/100
158/158 [==============================] - 3s 21ms/step - loss: 1.9917 - accuracy: 0.4095 - val_loss: 1.8647 - val_accuracy: 0.5315
Epoch 7/100
158/158 [==============================] - 3s 21ms/step - loss: 2.0332 - accuracy: 0.4238 - val_loss: 1.2133 - val_accuracy

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 100, 126)         504       
 hNormalization)                                                 
                                                                 
 conv1d (Conv1D)             (None, 100, 64)           24256     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 64)           0         
 )                                                               
                                                                 
 batch_normalization_3 (Batc  (None, 50, 64)           256       
 hNormalization)                                                 
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 128)           24704     
                                                      

In [19]:
model.save('VGG16.h5')

In [20]:
model.load_weights('VGG16.h5')

In [21]:
yhat = model.predict(x_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Train Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

40/40 [==============================] - 1s 5ms/step
Train Accuracy :-> 
87.6984126984127


In [22]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Test Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

17/17 [==============================] - 0s 7ms/step
Test Accuracy :-> 
79.07407407407408
